In [2]:
import warnings
warnings.filterwarnings("ignore")
from utils import *
import pickle
with open('chile.pickle', 'rb') as handle:
    studies = pickle.load(handle)      # Saca la cosa
study = studies[0] #.trials.__len__()
plot_all_studies(studies, .8, 0, 'chile') 

In [2]:
#Quality control
OPBG = [ 'age at diagnosis (months)',            
            'ntbc dosis mg/kg/day',
            'ntbc levels (dbs)',
            'sca (urine)',
            'methionine dbs',
            'tyrosine dbs',
            'phenylalanine dbs',
            'methionine (plasma)',
            'tyrosine (plasma)',
            'phenylalanine (plasma)',
            'inr',
            'pt (sec)',
            'bili total',
            'gpt',
            'got',
            'ggt',
            'alkaline phosphatase',
            'alfa-fetoprotein']
    
df1 = pd.read_csv('data/tirosinemia.csv').loc[:,OPBG]
df2 = pd.read_csv('data/tirosinemia_italia.csv').loc[:,OPBG]
compare_dataframes(df1, df2)
def extract_transform(path, keep_cols, cols_rows_with_missing_values, new_binary_target):
    _ , spark_df = SparkDataProcessor().process_data(
        data_csv_path= path,
        )

    return prepare_spark_df(
        spark_df=spark_df,
        keep_cols=keep_cols,
        cols_rows_with_missing_values=cols_rows_with_missing_values,
        new_binary_target = new_binary_target,
    )

binary_target = 'Alpha-Fet'
df_train = extract_transform("data/tirosinemia.csv", OPBG,  ['alfa-fetoprotein', 'sca (urine)', 'ntbc levels (dbs)'],binary_target)
df_test  = extract_transform("data/tirosinemia_italia.csv", OPBG,  ['alfa-fetoprotein', 'sca (urine)', 'ntbc levels (dbs)'],binary_target)

Both CSV files have the same number of columns: 18
Column names are the same.
Column order is the same.
All common columns have the same data types.
23/06/02 21:28:44 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 100.64.221.100 instead (on interface tailscale0)
23/06/02 21:28:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/06/02 21:28:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/02 21:28:47 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [3]:
filtered_trials = [
                    trial 
                    for study in studies
                    for trial in TwoObjectiveSolutions(study, auc_cutoff=0.8, s_cutoff=0).get_filtered_trials().filtered_trials
                ]

import warnings
from pandas.errors import PerformanceWarning


warnings.filterwarnings("ignore", category=PerformanceWarning)

import copy
AUCs = []
df_ranks_abnormal = pd.DataFrame()
df_ranks_healty = pd.DataFrame()



from tqdm import tqdm
from pandas.errors import PerformanceWarning

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=PerformanceWarning)
    for i, trial in tqdm(enumerate(copy.deepcopy(filtered_trials)), total=len(filtered_trials)):
        # print(trial.values)

        params_i = trial.params
        seed, kfold_splits, xgb_params = params_i.pop('seed'), params_i.pop('kfold_splits'), params_i

        model_instance = ModelInstance(
            df = df_train, 
            target = binary_target,
            xgb_params = xgb_params,
            kfold_splits = kfold_splits,
            Independent_testset = True,
            Independent_testset_df = df_test,
            seed = seed,
            
        )
        AUCs.append(model_instance.get_AUC_on_test_data())
        feature_metrics      = model_instance.get_feature_explanation()
        df_ranks_abnormal[i] = feature_metrics["SHAP_abnormal"]
        df_ranks_healty[i]   = feature_metrics["SHAP_healty"]



  0%|          | 0/17 [00:00<?, ?it/s]


AttributeError: 'tuple' object has no attribute 'select_dtypes'

In [18]:
import plotly.express as px
import numpy as np

df_ranks_long = df_ranks_abnormal.T
order = df_ranks_long.rank(axis="columns", ascending=False).median(axis="rows").sort_values().index
#df_ranks_long["auc_score"] = AUCs
df = df_ranks_long
import plotly.graph_objects as go

# Calculate the median of each variable and sort the columns in descending order
median_values = df.median().sort_values(ascending=False)
sorted_columns = median_values.index

# Create a list to store the box traces
box_traces = []

# Iterate over each sorted column
for column in sorted_columns:
    # Create a box trace for each variable
    box_trace = go.Box(
        y=df[column],
        name=column,
        marker_color='gray',  # Set the marker color to gray
        showlegend=False  # Hide the legend
    )
    # Add the box trace to the list
    box_traces.append(box_trace)

# Create the layout
layout = go.Layout(
    yaxis=dict(
        title='Importance'
    ),
    xaxis=dict(
        tickangle=45,  # Rotate the x-axis labels by 45 degrees
    ),
    autosize=False,  # Turn off autosizing
    width=800,  # Set the width of the figure
    height=300,  # Set the height of the figure
    margin=dict(
        l=0,  # Set left margin to 0
        r=0,  # Set right margin to 0
        t=0,  # Set top margin to 0
        b=0  # Set bottom margin to 0
    )
)

# Create the figure
fig = go.Figure(data=box_traces, layout=layout)

# Show the figure
fig.show()


In [4]:
import plotly.express as px
import numpy as np

df_ranks_long = df_ranks_abnormal.T
order = df_ranks_long.rank(axis="columns", ascending=False).median(axis="rows").sort_values().index
df_ranks_long["auc_score"] = AUCs

fig_barplot = px.bar(df_ranks_abnormal, barmode="group")

def rank_sort(df_ranks_abnormal: pd.DataFrame) -> pd.Index:
    """This takes the dataframe, ranks its variables, and sorts them by the mean ranking"""
    return df_ranks_abnormal.rank(axis="rows").mean(axis="columns").sort_values(ascending=False).index

order = rank_sort(df_ranks_abnormal) # Converts ranks to an ordered list

def wrapper_pcoorplot(df_ranks_abnormal, dims=order.tolist()):
    """Small wrapper around plotly parallel coordinates plot"""
    fig_ranks = px.parallel_coordinates(
        df_ranks_abnormal.T,
        dimensions=dims,
        color=df_ranks_long["auc_score"],
        # labels=labels_relabel,
        color_continuous_scale='BuGn',
    )

    # Calculate global min and max
    global_min = np.min(df_ranks_abnormal.values)
    global_max = np.max(df_ranks_abnormal.values)

    # Create a new 'range' key for each dimension in the figure
    for dimension in fig_ranks.data[0]['dimensions']:
        dimension['range'] = [global_min, global_max]

    fig_ranks.update_layout(
        yaxis_tickformat=".2f",
        coloraxis_colorbar=dict(title=f"{'AUC'} (test data)"),
    )

    return fig_ranks

# Creates the figure with the first [:15] features according to ranking
fig_abnormal = wrapper_pcoorplot(df_ranks_abnormal, order.tolist()[:10])
fig_abnormal.show()  # Show fig, as this is a notebook
